## Part 2. Portfolio analysis

## Part 2.1. Reading and choosing data

### Part 2.1.1. Importing libs

In [ ]:
import pandas as pd
import numpy as np
import random

from read_write_csv import read_csv, save_csv, save_json
from monte_carlo import simulate_portfolios

### Part 2.1.2. Reading the data

Read the data. read_csv function automatically convert Date column to datetime format and sets it as the index column

In [2]:
df = read_csv('historical_data.csv')

### Part 2.1.3. Data-check

Let us check how does the data look like at this point

In [3]:
df.describe()

,0P0000KQL0,0P0001I1JH,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,...,XEL,XOM,XON,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
count,1434.000000,1434.000000,2517.000000,2517.000000,2517.000000,2517.000000,2517.000000,0.0,0.0,2517.000000,...,2517.000000,2517.000000,0.0,2517.000000,2517.000000,2517.000000,2517.000000,2517.000000,2517.000000,2517.000000
mean,2.048117,2.087138,88.756119,27.945144,131.378494,92.384616,88.792928,NaN,NaN,76.214976,...,48.231286,66.775741,NaN,45.618223,76.444332,89.977027,115.963495,230.650095,36.991373,116.328325
std,0.221826,0.225107,40.293001,13.388271,41.063960,64.475861,43.802411,NaN,NaN,30.538865,...,13.741923,23.427996,NaN,10.558341,30.768412,29.955871,16.206040,140.109513,10.998845,56.333974
min,1.365500,1.385500,31.031214,9.040000,35.689999,20.697269,32.962017,NaN,NaN,30.864788,...,23.955330,25.031288,NaN,17.260000,27.042645,40.126244,75.456177,46.930000,15.628467,36.925087
25%,1.897600,1.940325,55.954002,14.670000,107.788757,34.155132,48.644257,NaN,NaN,42.275589,...,35.726334,54.053425,NaN,36.444599,48.644886,62.214317,105.209770,104.410004,27.387794,57.855518
50%,2.037350,2.075960,75.648575,27.388784,138.948288,62.786938,72.829781,NaN,NaN,77.427391,...,52.689938,58.233849,NaN,47.532429,73.231926,89.732719,113.299126,211.399994,37.348949,120.481628
75%,2.188475,2.230000,129.002914,40.008011,151.840866,149.165283,130.198578,NaN,NaN,104.471436,...,60.203667,79.988983,NaN,54.421982,100.557396,117.584152,125.037384,302.769989,44.652092,167.719940
max,2.615000,2.665100,175.479584,56.988728,224.340866,237.330002,203.869995,NaN,NaN,133.728104,...,72.919998,124.348221,NaN,65.587769,144.777924,141.782349,168.737976,614.549988,66.437309,240.630768


### Part 2.1.4. Processing Data

We have lot's of NaNs which is not good for further processing. We would like to drop the rows where there is at least 95% of NaNs and we would like to fill the other ones using linear interpolation

In [4]:
# here we can configure the desired threshold
threshold = 0.95

# then we drop rows where there is at least 95% of nans, by leaving the rows where there is less than 15% of nans
df = df[df.isnull().mean(axis=1) < threshold]

# the rows where there is less than 15% of nans, we fill using linear interpolation
df = df.interpolate(
    method='linear', axis=0, limit_direction='forward', inplace=False
    )

# dataframe

# there can be still some nans, and we would like simply fill them
if df.isnull().any().any():
    df = df.ffill().bfill()
    df = df.dropna(axis=0, how='all').dropna(axis=1, how='all')

df

,0P0000KQL0,0P0001I1JH,A,AAL,AAP,AAPL,ABBV,ABT,ACGL,ACN,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2014-12-01,1.8539,1.87541,38.278641,45.366467,132.242096,25.625433,45.692089,36.629391,18.070240,72.905159,...,151.146072,25.016867,59.732052,49.713570,33.862663,46.250717,99.957275,72.900002,21.246679,41.264732
2014-12-02,1.8539,1.87541,38.637589,45.347519,133.509888,25.527447,45.797863,37.373215,18.260422,72.888245,...,150.929489,25.301571,60.922173,50.203979,33.492672,46.836475,102.611519,75.150002,21.457123,41.608845
2014-12-03,1.8539,1.87541,38.867683,45.707565,136.711029,25.816950,45.288784,37.753391,18.399887,73.345978,...,146.875870,25.389166,61.413742,50.540016,34.259071,46.406128,101.912102,77.080002,21.714323,41.739048
2014-12-04,1.8539,1.87541,38.904484,47.043552,137.798965,25.718969,45.996212,37.596367,18.472788,73.193390,...,141.115799,25.411076,61.038605,50.503681,33.977177,46.459900,102.190086,76.550003,21.683151,40.920635
2014-12-05,1.8539,1.87541,38.932106,48.332153,138.635254,25.609850,46.088776,37.439342,18.520334,73.066261,...,141.964706,25.206671,60.682873,50.885124,33.994804,46.800610,103.086792,76.489998,22.291088,40.902035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-22,2.0151,2.06870,133.839996,14.380000,40.790001,229.869995,176.949997,117.760002,99.690002,358.660004,...,91.290001,71.349998,121.790001,18.770000,126.515785,135.270004,110.220001,397.190002,60.580002,176.960007
2024-11-25,2.0370,2.09130,134.490005,14.920000,43.419998,232.869995,177.059998,118.129997,100.059998,361.290009,...,93.449997,71.599998,119.970001,19.520000,128.580002,137.050003,110.959999,400.940002,61.730000,178.710007
2024-11-26,2.0631,2.11800,134.369995,14.640000,43.840000,235.059998,181.139999,117.959999,100.410004,363.179993,...,91.860001,71.870003,117.970001,19.190001,124.540001,138.630005,110.639999,402.450012,61.110001,175.699997


### Part 2.1.5. Random data selection

If we would go further and leave such amount of stocks, we would receive a bullshit result in the end. That happens because if we have, let's say, 100 stocks in portfolio, then its overdiversified. It isn't even possible to have such amount of stocks in portfolio, as it is really not so effective as in case of choosing few stocks. We would buy index (sp100 for instance) as a stock and would not need to do all that stuff. That is why let's consider only some of the stocks and do it in a random manner

In [5]:
# here we would like a user to specify the number of stocks to analyze
def prompt_amount_of_stocks_to_consider(df):
    # we need to exclude the date column from the number of stocks, as date is not a stock basically :)
    maximum_allowed = len(df.columns) - 1  
    while True:
        try:
            prompted_value = int(input(f'Number of stocks? (no more than {maximum_allowed}): '))
            if 0 < prompted_value <= maximum_allowed:
                return prompted_value
            else:
                print(f"Enter a correct value (1 to {maximum_allowed})!")
        except ValueError:
            print("Invalid input. Please enter an integer.")

# simply call a function to store prompted number of stocks in a variable
num_of_stocks = prompt_amount_of_stocks_to_consider(df)

# here we calculate the number of stocks to drop from df
number_of_stocks_to_drop = len(df.columns) - num_of_stocks

""" then we should set the columns that shouldn't be excluded in any case
 - like the date column. So the variable column_to_exclude will contain the 
 columns' names (or tickers of stocks) that can be potentially dropped"""
columns_to_exclude = [col for col in df.columns if col != 'Date']  

# then we randomly choose which columns to drop 
if number_of_stocks_to_drop > 0:
    columns_to_drop = random.sample(columns_to_exclude, number_of_stocks_to_drop)
    df = df.drop(columns=columns_to_drop)

# now we can check what tickers do we have in the end and the total number of tickers
final_tickers = [col for col in df.columns if col != 'Date']  

print(f"Final number of tickers: {len(final_tickers)}")

Final number of tickers: 5


### Part 2.1.6. Autosave

We wouldn't like to go through all of this processes each time, and we would like to have a kind of an autosave to be able to easily make changes and to revert if there is such a need. So, we will do next:

In [6]:
save_csv(df, 'save216.csv')

'c:\\Users\\nikit\\Desktop\\Personal\\pythonLanguage\\portfolio_optimization_ml\\src\\data\\save216.csv'

## 2.2. Analysis

### 2.2.1. Average Prices 

As we have lots of tickers to analyze, we couldn't plot them on the linear graph all together to get the global picture. For that, we would like to make a column that is going to contain all the values brought together in the average form.

In [7]:
avg_price = pd.DataFrame()
avg_price.index = df.index
avg_price['Average Price'] = df.mean(axis=1)
avg_price.tail()

,Average Price
Date,
2024-11-22,285.475768
2024-11-25,287.823072
2024-11-26,288.314812
2024-11-27,286.804999
2024-11-29,287.912006


Also, we would like to export this file to plot the graph later on.

In [8]:
save_csv(avg_price, 'avg_price.csv')

'c:\\Users\\nikit\\Desktop\\Personal\\pythonLanguage\\portfolio_optimization_ml\\src\\data\\avg_price.csv'

### 2.2.2 Average Returns

Then we can iterate through the dataframe using the list of tickers to apply pct_change method to calculate the returns

In [9]:
# here we create new df for returns
returns = pd.DataFrame()  

# then we iterate through the final tickers we got for the analysis
for ticker in final_tickers:
    returns[ticker+' Returns'] = df[ticker].pct_change()  # apply pct_change method to calculate returns

returns.tail()

,KO Returns,MCD Returns,MOH Returns,NEE Returns,PH Returns
Date,,,,,
2024-11-22,0.002509,0.006274,-0.017834,-0.010996,0.005963
2024-11-25,0.007196,0.020360,0.024691,0.008421,-0.003466
2024-11-26,0.002641,0.000473,-0.003792,0.011091,0.003450
2024-11-27,-0.001859,-0.004218,-0.000808,0.013292,-0.009861
2024-11-29,0.002111,0.003152,0.004383,0.001910,0.004315


As we face same situtation as in the previous block - the amount of stocks doesn't allow us to make a separate chart for each particular stock, here we wouldn't just simply plot the histogram for the returns. That is why we, first of all, would like to make a column with the average returns for all the stocks, and then plot the graph itself. 

In [10]:
avg_returns = pd.DataFrame()
avg_returns['Average Return for Stocks'] = returns.mean(axis=1)
avg_returns.head()

,Average Return for Stocks
Date,
2014-12-01,NaN
2014-12-02,0.002241
2014-12-03,0.004753
2014-12-04,-0.008293
2014-12-05,0.002390


Then we'd like to save it to plot the graph later on

In [11]:
save_csv(avg_returns, 'avg_returns.csv')

'c:\\Users\\nikit\\Desktop\\Personal\\pythonLanguage\\portfolio_optimization_ml\\src\\data\\avg_returns.csv'

### 2.2.3. Some additional calculations

For further calculations - i.e. for Monte Carlo simulations we have to find mean table and covariance table for the returns, considering the fact that there is 252 trading days in a year.

In [12]:
trading_days = 252

In [13]:
mean_table = (returns.mean()*trading_days).to_numpy()
covariance_table = (returns.cov()*trading_days).to_numpy()

### <mark>2.2.4. Monte Carlo Simulation</mark>

Monte Carlo simulation is a good method to assess the potential outcomes of different investment strategies or different allocations under different conditions. We will run a simulation on different allocations of the same stocks to find the best optimum allocation

In [14]:
""" that one stands for the number of simulations that are going to be run. It is
better to have as much as possible - this will only affect the performance of
charts rendering, as there will be lots of points to render, which is very demanding.
Recommended value is 100.000 per stocks"""
num_simulations = len(final_tickers) * 100_000

# amount of stocks we have in portfolio that is analyzed
num_assets = len(final_tickers)  

# and we are also going to declare some additional variables for further computation
all_weights = np.zeros((num_simulations, num_assets))
returns_array = np.zeros(num_simulations)
volatilities_array = np.zeros(num_simulations)
sharpe_array = np.zeros(num_simulations)

We should also set risk-free-rate here. It is usally 10y us bonds treasury percent

In [15]:
risk_free_rate = 0.046

Now is the Monte Carlo's turn. We pass the variables declared above, and Monte Carlo will give random outputs with the given variables. As result, we will have arrays with the data with different Sharpe ratios. We will reference to sharpe_array to find the maximum sharpe, then we will access other arrays using the maximum sharpe to find the optimal results.

In [16]:
simulate_portfolios(
    num_simulations,
    num_assets,
    mean_table,
    covariance_table,
    risk_free_rate,
    all_weights,
    returns_array,
    volatilities_array,
    sharpe_array
)

Then we should find a sharpe ratio that is the highest one and we will use it to access the metrics, related to this sharpe - so the metrics that gave us such a sharpe

In [17]:
max_sharpe = sharpe_array.max()
i_max_sharpe = sharpe_array.argmax()
all_weights = all_weights[i_max_sharpe, :]

volatility_max_sharpe = volatilities_array[i_max_sharpe]
returns_max_sharpe = returns_array[i_max_sharpe]

print(f'Volatility and Returns of portfolio with the max sharpe: {volatility_max_sharpe, returns_max_sharpe}')

Volatility and Returns of portfolio with the max sharpe: (np.float64(0.19973192949101623), np.float64(0.20324333746399698))


Now let us save the result as json. _This is going to take lots of memory space_

In [18]:
monte_carlo_data = {
    'volatilities_array': volatilities_array.tolist(),
    'returns_array': returns_array.tolist(),
    'sharpe_array': sharpe_array.tolist(),
    'volatility_max_sharpe': volatility_max_sharpe.tolist(),
    'returns_max_sharpe': returns_max_sharpe.tolist(),
    'num_stocks': final_tickers,
    'num_simulations': num_simulations,
    'all_weights': all_weights.tolist()
}

In [19]:
save_json(monte_carlo_data, 'monte_carlo_data.json')